In [1]:
from datetime import date
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 5)

df_records_raw = catalog.load('raw/oai/records#parquet')

[12/07/25 20:03:48] INFO     Loading data from raw/oai/records#parquet (ParquetDataset)...     ]8;id=874627;file:///home/pablo/dev/scholar/kedro-scholar/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=392127;file:///home/pablo/dev/scholar/kedro-scholar/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#1048\1048]8;;\

In [2]:
df_records_raw

,record_id,datestamp,set_id,col_id,title,date_issued,creators,types,identifiers,languages,subjects,publishers,relations,rights,extract_datetime
0,oai:ri.conicet.gov.ar:11336/179477,2024-01-12T04:51:20Z,"[com_11336_35, com_11336_14, col_11336_36, snrd]",com_11336_35,Paisajes mesetarios en Patagonia: Tecnología d...,2016-12,"[Cassiodoro, Gisela Eva]","[info:eu-repo/semantics/article, info:ar-repo/...","[http://hdl.handle.net/11336/179477, Cassiodor...",[spa],"[MESETAS, TECNOLOGÍA, USO DEL ESPACIO, LOGÍSTI...",[Instituto Nacional de Antropología y Pensamie...,[info:eu-repo/semantics/altIdentifier/url/http...,"[info:eu-repo/semantics/openAccess, https://cr...",2025-12-05 00:00:00+00:00
1,oai:ri.conicet.gov.ar:11336/116972,2020-10-27T20:15:28Z,"[com_11336_219, com_11336_218, col_11336_89919...",com_11336_219,Distancias entre la ecología y la praxis ambie...,2011,"[Nuñez, Paula Gabriela]","[info:eu-repo/semantics/publishedVersion, info...","[http://hdl.handle.net/11336/116972, Nuñez, Pa...",[spa],"[ECOLOGÍA, PRAXIS AMBIENTAL, ECOFEMINISMO, TEO...",[Universidad Nacional de La Plata],[info:eu-repo/semantics/altIdentifier/url/http...,"[info:eu-repo/semantics/openAccess, https://cr...",2025-12-05 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252664,oai:ri.conicet.gov.ar:11336/75430,2019-05-02T18:30:25Z,"[com_11336_252, com_11336_231, col_11336_253, ...",com_11336_252,"Caretas y pecadores: ajustes secundarios, gube...",2017-10,"[Manchado, Mauricio Carlos]","[info:eu-repo/semantics/article, info:ar-repo/...","[http://hdl.handle.net/11336/75430, Manchado, ...",[spa],"[RESISTENCIAS, PRISIÓN, PENTECOSTALISMO, AJUST...",[Centro de Estudios e Investigaciones Laborales],[info:eu-repo/semantics/altIdentifier/url/http...,"[info:eu-repo/semantics/openAccess, https://cr...",2025-12-05 00:00:00+00:00
252665,oai:ri.conicet.gov.ar:11336/264054,2025-06-17T09:25:18Z,"[com_11336_71, com_11336_50, col_11336_116696,...",com_11336_71,Composting and on-site reusing of pear pomace,2023,"[Iturmendi, Facundo, Bongiovani, Natalia Soled...","[info:eu-repo/semantics/publishedVersion, info...","[http://hdl.handle.net/11336/264054, Compostin...",[eng],"[COMPOSTING, PEAR, POMACE, https://purl.org/be...",[Asociación Argentina de Ingenieros Químicos],[info:eu-repo/semantics/altIdentifier/url/http...,"[info:eu-repo/semantics/openAccess, https://cr...",2025-12-05 00:00:00+00:00


In [3]:
def _pick_load_dt(df: pd.DataFrame):
    # Si hay una sola fecha en el batch, usala; si hay varias, quedate con la más reciente;
    # si no hay, hoy.
    if 'load_datetime' not in df.columns or df['load_datetime'].isna().all():
        return date.today()
    vals = df['load_datetime'].dropna()
    if vals.nunique() == 1:
        return vals.iloc[0]
    return pd.to_datetime(vals).max().date()


In [7]:
def oai_load_records(df_records_raw: pd.DataFrame, env = 'dev')-> pd.DataFrame:

    df_records_raw = df_records_raw.copy()
    load_dt = _pick_load_dt(df_records_raw)

    if env == 'dev':
        df_records_raw = df_records_raw.head(1000)

    def _select(columns):
        return df_records_raw.loc[:, columns].copy()

    def _explode(column):
        return (
            _select(['record_id', column, 'extract_datetime'])
            .explode(column, ignore_index=True)
            .assign(load_datetime=load_dt)
        )

    df_records = _select(['record_id','col_id','title','date_issued', 'extract_datetime']).assign(load_datetime=load_dt)
    df_record_creators = _explode('creators')
    df_record_types = _explode('types')
    df_record_identifiers = _explode('identifiers')
    df_record_languages = _explode('languages')
    df_record_subjects = _explode('subjects')
    df_record_publishers = _explode('publishers')
    df_record_relations = _explode('relations')
    df_record_rights = _explode('rights')
    df_record_sets = _explode('set_id')

#    df_record_sets = _select(['record_id','set_id', 'extract_datetime'])
#    sets_df = df_record_sets.pop('set_id').apply(pd.Series)
#    sets_df = sets_df.rename(columns=lambda i: f'set_{i}')
#    df_record_sets = pd.concat([df_record_sets, sets_df], axis=1)
#    df_record_sets['load_datetime'] = load_dt

    return df_records, df_record_creators, df_record_types, df_record_identifiers, df_record_languages, df_record_subjects, df_record_publishers, df_record_relations, df_record_rights, df_record_sets


In [8]:
df_records, df_record_creators, df_record_types, \
    df_record_identifiers, df_record_languages, df_record_subjects, \
        df_record_publishers, df_record_relations, df_record_rights, \
             df_record_sets = oai_load_records(df_records_raw)

In [9]:
df_records

,record_id,col_id,title,date_issued,extract_datetime,load_datetime
0,oai:ri.conicet.gov.ar:11336/179477,com_11336_35,Paisajes mesetarios en Patagonia: Tecnología d...,2016-12,2025-12-05 00:00:00+00:00,2025-12-07
1,oai:ri.conicet.gov.ar:11336/116972,com_11336_219,Distancias entre la ecología y la praxis ambie...,2011,2025-12-05 00:00:00+00:00,2025-12-07
...,...,...,...,...,...,...
998,oai:ri.conicet.gov.ar:11336/248879,com_11336_333,Magnetic materials in oil industry,2022,2025-12-05 00:00:00+00:00,2025-12-07
999,oai:ri.conicet.gov.ar:11336/124798,com_11336_123,Ternary copper(II) complex of 5-hydroxytryptop...,2019-12,2025-12-05 00:00:00+00:00,2025-12-07


In [10]:
df_record_creators


,record_id,creators,extract_datetime,load_datetime
0,oai:ri.conicet.gov.ar:11336/179477,"Cassiodoro, Gisela Eva",2025-12-05 00:00:00+00:00,2025-12-07
1,oai:ri.conicet.gov.ar:11336/116972,"Nuñez, Paula Gabriela",2025-12-05 00:00:00+00:00,2025-12-07
...,...,...,...,...
4382,oai:ri.conicet.gov.ar:11336/124798,"Ferrer, Evelina Gloria",2025-12-05 00:00:00+00:00,2025-12-07
4383,oai:ri.conicet.gov.ar:11336/124798,"Williams, Patricia Ana María",2025-12-05 00:00:00+00:00,2025-12-07


In [11]:
df_record_types


,record_id,types,extract_datetime,load_datetime
0,oai:ri.conicet.gov.ar:11336/179477,info:eu-repo/semantics/article,2025-12-05 00:00:00+00:00,2025-12-07
1,oai:ri.conicet.gov.ar:11336/179477,info:ar-repo/semantics/artículo,2025-12-05 00:00:00+00:00,2025-12-07
...,...,...,...,...
3111,oai:ri.conicet.gov.ar:11336/124798,info:ar-repo/semantics/artículo,2025-12-05 00:00:00+00:00,2025-12-07
3112,oai:ri.conicet.gov.ar:11336/124798,info:eu-repo/semantics/publishedVersion,2025-12-05 00:00:00+00:00,2025-12-07


In [12]:
df_record_identifiers

,record_id,identifiers,extract_datetime,load_datetime
0,oai:ri.conicet.gov.ar:11336/179477,http://hdl.handle.net/11336/179477,2025-12-05 00:00:00+00:00,2025-12-07
1,oai:ri.conicet.gov.ar:11336/179477,"Cassiodoro, Gisela Eva; Paisajes mesetarios en...",2025-12-05 00:00:00+00:00,2025-12-07
...,...,...,...,...
5063,oai:ri.conicet.gov.ar:11336/124798,CONICET Digital,2025-12-05 00:00:00+00:00,2025-12-07
5064,oai:ri.conicet.gov.ar:11336/124798,CONICET,2025-12-05 00:00:00+00:00,2025-12-07


In [13]:
df_record_languages

,record_id,languages,extract_datetime,load_datetime
0,oai:ri.conicet.gov.ar:11336/179477,spa,2025-12-05 00:00:00+00:00,2025-12-07
1,oai:ri.conicet.gov.ar:11336/116972,spa,2025-12-05 00:00:00+00:00,2025-12-07
...,...,...,...,...
998,oai:ri.conicet.gov.ar:11336/248879,eng,2025-12-05 00:00:00+00:00,2025-12-07
999,oai:ri.conicet.gov.ar:11336/124798,eng,2025-12-05 00:00:00+00:00,2025-12-07


In [14]:
df_record_subjects


,record_id,subjects,extract_datetime,load_datetime
0,oai:ri.conicet.gov.ar:11336/179477,MESETAS,2025-12-05 00:00:00+00:00,2025-12-07
1,oai:ri.conicet.gov.ar:11336/179477,TECNOLOGÍA,2025-12-05 00:00:00+00:00,2025-12-07
...,...,...,...,...
6314,oai:ri.conicet.gov.ar:11336/124798,https://purl.org/becyt/ford/1.4,2025-12-05 00:00:00+00:00,2025-12-07
6315,oai:ri.conicet.gov.ar:11336/124798,https://purl.org/becyt/ford/1,2025-12-05 00:00:00+00:00,2025-12-07


In [15]:
df_record_publishers


,record_id,publishers,extract_datetime,load_datetime
0,oai:ri.conicet.gov.ar:11336/179477,Instituto Nacional de Antropología y Pensamien...,2025-12-05 00:00:00+00:00,2025-12-07
1,oai:ri.conicet.gov.ar:11336/116972,Universidad Nacional de La Plata,2025-12-05 00:00:00+00:00,2025-12-07
...,...,...,...,...
998,oai:ri.conicet.gov.ar:11336/248879,Elsevier,2025-12-05 00:00:00+00:00,2025-12-07
999,oai:ri.conicet.gov.ar:11336/124798,Elsevier Science Inc,2025-12-05 00:00:00+00:00,2025-12-07


In [16]:
df_record_relations


,record_id,relations,extract_datetime,load_datetime
0,oai:ri.conicet.gov.ar:11336/179477,info:eu-repo/semantics/altIdentifier/url/https...,2025-12-05 00:00:00+00:00,2025-12-07
1,oai:ri.conicet.gov.ar:11336/116972,info:eu-repo/semantics/altIdentifier/url/https...,2025-12-05 00:00:00+00:00,2025-12-07
...,...,...,...,...
1718,oai:ri.conicet.gov.ar:11336/124798,info:eu-repo/semantics/altIdentifier/doi/10.10...,2025-12-05 00:00:00+00:00,2025-12-07
1719,oai:ri.conicet.gov.ar:11336/124798,info:eu-repo/semantics/altIdentifier/url/https...,2025-12-05 00:00:00+00:00,2025-12-07


In [17]:
df_record_rights

,record_id,rights,extract_datetime,load_datetime
0,oai:ri.conicet.gov.ar:11336/179477,info:eu-repo/semantics/openAccess,2025-12-05 00:00:00+00:00,2025-12-07
1,oai:ri.conicet.gov.ar:11336/179477,https://creativecommons.org/licenses/by-nc/2.5...,2025-12-05 00:00:00+00:00,2025-12-07
...,...,...,...,...
2010,oai:ri.conicet.gov.ar:11336/124798,info:eu-repo/semantics/restrictedAccess,2025-12-05 00:00:00+00:00,2025-12-07
2011,oai:ri.conicet.gov.ar:11336/124798,https://creativecommons.org/licenses/by-nc-sa/...,2025-12-05 00:00:00+00:00,2025-12-07


In [18]:
df_record_sets

,record_id,set_id,extract_datetime,load_datetime
0,oai:ri.conicet.gov.ar:11336/179477,com_11336_35,2025-12-05 00:00:00+00:00,2025-12-07
1,oai:ri.conicet.gov.ar:11336/179477,com_11336_14,2025-12-05 00:00:00+00:00,2025-12-07
...,...,...,...,...
4541,oai:ri.conicet.gov.ar:11336/124798,col_11336_124,2025-12-05 00:00:00+00:00,2025-12-07
4542,oai:ri.conicet.gov.ar:11336/124798,snrd,2025-12-05 00:00:00+00:00,2025-12-07
